# Query the EFD

Craig Lage - 06-Aug-24

In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.daf.butler import Butler

In [ ]:
client = makeEfdClient()
butler = Butler('LSSTCam', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded'])

In [ ]:
dayObss = [20250718, 20250812, 20250915, 20250921, 20251022, 20251022]
wfs = ['SW0', 'SW1']
scis = ['S00', 'S01', 'S02', 'S10', 'S11', 'S12','S20', 'S21', 'S22']
print("All times are from end of exposure")
print()
for i, dayObs in enumerate(dayObss):
    dayStr = dayObsIntToString(dayObs)
    nextDayObs = calcNextDay(dayObs)
    nextDayStr = dayObsIntToString(nextDayObs)
    start = Time(f"{dayStr} 12:00:00Z", scale='utc')
    end = Time(f"{nextDayStr} 12:00:00Z", scale='utc')
    oods = getEfdData(client, "lsst.sal.MTOODS.logevent_imageInOODS", begin=start, end=end)
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    #print(dayObs, len(exposureList))
    first_wf = []
    last_wf = []
    wf_delta = []
    first_sci = []
    last_sci = []
    sci_delta = []
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science', 'dark']:
            continue
        try:
            seqNum = record.seq_num
            if i == 5 and seqNum < 21:
                continue
            endExposure = record.timespan.end.unix_tai
            obsid = f"MC_O_{dayObs}_{seqNum:06d}"
            this_image = oods[oods['obsid'] == obsid]
            wavefronts = this_image[this_image['sensor'].isin(wfs)]
            sciences = this_image[this_image['sensor'].isin(scis)]
            wf_times = wavefronts['private_kafkaStamp'].values - endExposure
            first_wf.append(np.min(wf_times))
            last_wf.append(np.max(wf_times))
            wf_delta.append(np.max(wf_times) - np.min(wf_times))
            sci_times = sciences['private_kafkaStamp'].values - endExposure
            first_sci.append(np.min(sci_times))
            last_sci.append(np.max(sci_times))
            sci_delta.append(np.max(sci_times) - np.min(sci_times))
        except:
            continue
    if i == 5:
        print(f"{dayObs}, seqNum > 20")
    else:
        print(dayObs)
    print(f"Median time to last wavefront = {np.median(last_wf):.1f} seconds")
    print(f"Worst case time to last wavefront = {np.max(last_wf):.1f} seconds")
    print(f"Median delay from first to last wavefront = {np.median(wf_delta):.1f} seconds")
    print(f"Median time to last science = {np.median(last_sci):.1f} seconds")
    print(f"Worst case time to last science = {np.max(last_sci):.1f} seconds")
    print(f"Median delay from first to last science = {np.median(sci_delta):.1f} seconds")
    print()
        

In [ ]:
client = makeEfdClient()
butler = Butler('LATISS', collections=['LATISS/raw/all'])

In [ ]:
dayObs = 20251112
print("All times are from end of exposure")
print()

dayStr = dayObsIntToString(dayObs)
nextDayObs = calcNextDay(dayObs)
nextDayStr = dayObsIntToString(nextDayObs)
start = Time(f"{dayStr} 12:00:00Z", scale='utc')
end = Time(f"{nextDayStr} 12:00:00Z", scale='utc')
oods = getEfdData(client, "lsst.sal.ATOODS.logevent_imageInOODS", begin=start, end=end)
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
            where=f"exposure.day_obs={dayObs} and instrument='LATISS'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
seqNums = []
ingest_times = []
for [id,record] in exposureList:
    if record.observation_type not in ['acq', 'science', 'cwfs']:
        continue
    try:
        seqNum = record.seq_num
        endExposure = record.timespan.end.unix_tai
        obsid = f"AT_O_{dayObs}_{seqNum:06d}"
        image = oods[oods['obsid'] == obsid]
        ingest_time = image['private_kafkaStamp'].values[0] - endExposure
        seqNums.append(seqNum)
        ingest_times.append(ingest_time)
    except:
        continue
        

In [ ]:
len(ingest_times)

In [ ]:
plt.title(f"AuxTel image ingestion times {dayObs}")
plt.scatter(seqNums, ingest_times)
plt.xlabel("Sequence number")
plt.ylabel("Time from shutter close to image ingestion (sec)")
plt.savefig(f"/home/cslage/DATA/Image_Ingestion_Delays_{dayObs}.png")